In [1]:
!pwd
import sys

%reload_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext lab_black
sys.executable

/c/Users/81908/jupyter_notebook/pytorch_lightning_work/kaggle_Cassava/notebook/stacking_test/code_test


'C:\\Users\\81908\\Anaconda3\\envs\\lightning\\python.exe'

In [2]:
import sys

sys.path.append(r"C:\Users\81908\MyGitHub\kaggle_Cassava\code")
sys.path.append(r"C:\Users\81908\Git\microsoftvision")
import microsoftvision

In [3]:
import os
import cv2
import glob
import json
import math
import yaml
import pickle
import random
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, log_loss
from tqdm.notebook import tqdm
import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import timm
import pytorch_lightning as pl
from efficientnet_pytorch import EfficientNet
from deit_models import deit_base_patch16_224
import microsoftvision
import warnings

warnings.filterwarnings("ignore")

In [4]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)
num_workers = 0
n_classes = 5

device: cuda


In [5]:
from params import base_data

df = base_data.df
test_df = base_data.test_df
name_mapping = base_data.name_mapping
path = base_data.ROOT_DIR
noise_image_id = base_data.noise_image_id

In [6]:
test_preds_list = [
    pd.DataFrame([0.015411, 0.01225, 0.287251, 0.006122, 0.678966]).T,
    pd.DataFrame([0.013555, 0.057926, 0.476243, 0.00123, 0.451045]).T,
    pd.DataFrame([0.034041, 0.021128, 0.717434, 0.089816, 0.137581]).T,
    pd.DataFrame([0.015216, 0.039199, 0.549125, 0.012901, 0.38356]).T,
    pd.DataFrame([0.052416, 0.016144, 0.262874, 0.027701, 0.640865]).T,
]
test_preds_list

[          0        1         2         3         4
 0  0.015411  0.01225  0.287251  0.006122  0.678966,
           0         1         2        3         4
 0  0.013555  0.057926  0.476243  0.00123  0.451045,
           0         1         2         3         4
 0  0.034041  0.021128  0.717434  0.089816  0.137581,
           0         1         2         3        4
 0  0.015216  0.039199  0.549125  0.012901  0.38356,
           0         1         2         3         4
 0  0.052416  0.016144  0.262874  0.027701  0.640865]

In [7]:
import sys

sys.path.append(r"C:\Users\81908\MyGitHub\kaggle_Cassava\code")
from lightning_cassava_stacking import (
    train_stacking,
    pred_stacking,
    StackingConfig,
)
from lightning_cassava_pred import set_seed

In [8]:
# noise_idx = None
noise_idx = list(df[df["image_id"].isin(noise_image_id)].index)
# noise_idx

In [9]:
# cnmn2d params
StackingCFG = StackingConfig()
StackingCFG.device = device
StackingCFG.num_workers = num_workers
StackingCFG.n_classes = n_classes
StackingCFG.arch = "cnmn2d"

## 20210212_objective11
# StackingCFG.cnn2d_params = {
#    "kwargs_head": {
#        "drop_rate": 0.2,
#        "n_features_list": [-1, 5],
#        "use_bn": True,
#        "use_tail_as_out": True,
#        "use_wn": True,
#    },
#    "n_channels_list": [1, 28],
#    "n_classes": n_classes,
#    "n_models": 5,
#    "use_bias": False,
# }
# StackingCFG.weight_decay = 0.001
# StackingCFG.smoothing = 0.2
# StackingCFG.t1 = 0.9
# StackingCFG.t2 = 1.0
# StackingCFG.gauss_scale = 0.0
# StackingCFG.cutmix_p = 0.0
# StackingCFG.alpha = 5.0
# StackingCFG.seeds = list(range(2))
# s_type = 1
gauss_scale = 0.0
is_noise_only_test = False
is_del_df_noise_image_id = False
pseudo_th = 0.0


# oof=0.9057811842781698(tta入れた)  #6
StackingCFG.cnn2d_params = {
    "kwargs_head": {
        "drop_rate": 0.8,
        "n_features_list": [-1, 3, 5],
        "use_bn": False,
        "use_tail_as_out": True,
        "use_wn": False,
    },
    "n_channels_list": [1, 12],
    "n_classes": 5,
    "n_models": 5,
    "use_bias": True,
}
StackingCFG.train_n_classes = 5
StackingCFG.weight_decay = 1e-05
StackingCFG.smoothing = 0.3
StackingCFG.t1 = 0.7
StackingCFG.t2 = 1.3
StackingCFG.gauss_scale = 0.24
StackingCFG.cutmix_p = 0.0
StackingCFG.alpha = 1.0
# StackingCFG.seeds = list(range(2))
StackingCFG.seeds = list(range(1))
StackingCFG.undersample_rate = 0.4  # cls3をundersampleing
s_type = 1
gauss_scale = 0
is_noise_only_test = False

# ノイズかどうか分類するモデルにするか
# is_make_noise_model = False
is_make_noise_model = True
if is_make_noise_model:
    StackingCFG.cnn2d_params["kwargs_head"]["n_features_list"] = [-1, 2]
    StackingCFG.train_n_classes = 2
    # StackingCFG.device = "cpu"
    s_type = 0  # pseudo_labelは入れてはダメ
    StackingCFG.lr = 0.1
    StackingCFG.min_lr = StackingCFG.lr * 0.01
    StackingCFG.monitor = "val_loss"
    StackingCFG.batch_size = 256  # // 4
    StackingCFG.n_over = 10

if gauss_scale > 0.0:
    set_seed(seed=42)

print(StackingCFG.__dict__)

{'seeds': [0], 'n_classes': 5, 'train_n_classes': 2, 'max_epochs': 200, 'patience': 50, 'n_splits': 5, 'shuffle': True, 'batch_size': 256, 'accumulate_grad_batches': 1, 'opt': 'adam', 'lr_scheduler': 'CosineAnnealingWarmRestarts', 'T_max': 200, 'T_0': 200, 'lr': 0.1, 'min_lr': 0.001, 'weight_decay': 1e-05, 'smoothing': 0.3, 'train_loss_name': 'BiTemperedLoss', 't1': 0.7, 't2': 1.3, 'monitor': 'val_loss', 'out_dir': '.', 'arch': 'cnmn2d', 'mlp_params': None, 'cnn1d_params': None, 'cnn2d_params': {'kwargs_head': {'drop_rate': 0.8, 'n_features_list': [-1, 2], 'use_bn': False, 'use_tail_as_out': True, 'use_wn': False}, 'n_channels_list': [1, 12], 'n_classes': 5, 'n_models': 5, 'use_bias': True}, 'gauss_scale': 0.24, 'cutmix_p': 0.0, 'alpha': 1.0, 'cls3_undersample_rate': 0.0, 'n_over': 10, 'device': 'cuda', 'num_workers': 0, 'undersample_rate': 0.4}


In [10]:
StackingCFG.max_epochs = 5
StackingCFG.T_max = StackingCFG.max_epochs
StackingCFG.T_0 = StackingCFG.max_epochs

In [11]:
%%time
# cnmn2d semi supervised learning
StackingCFG.out_dir = StackingCFG.arch
os.makedirs(StackingCFG.out_dir, exist_ok=True)


# load oof
preds = []

# --------------------------------------------------------------------------------------------
# ---- anonamename ----
#tta_dir = "../input/cassava-efficientnetwithpytorchlightning/kaggle_upload_oof_tta"
tta_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\notebook\check_oof\cassava-emsemble-v2_tta_oof\kaggle_upload_oof_tta"

pkl = f"{tta_dir}/tf_efficientnet_b4_ns_tta.pkl"
pred = pickle.load(open(pkl, "rb"))
preds.append(pred.values)

pkl = f"{tta_dir}/byol_seresnext50_32x4d_cutmix_labelsmooth_half_tta.pkl"
pred = pickle.load(open(pkl, "rb"))
preds.append(pred.values)

pkl = f"{tta_dir}/vit_b16_224_fold10_tta.pkl"
pred = pickle.load(open(pkl, "rb"))
preds.append(pred.values)

#pkl = f"{tta_dir}/resnest101e_cleanlab_noise_cutmix_tta.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{tta_dir}/resnest101e_tta.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{tta_dir}/tf_efficientnet_b4_ns_fold3_tta.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{tta_dir}/vit_base_patch32_384.fit_tta.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

# ---- SiNpcw ----
#m_dir = "../input/cassavapkl"
m_dir = r"C:\Users\81908\jupyter_notebook\pytorch_lightning_work\kaggle_Cassava\kaggle_datasets_dl\cassavapkl_v20"

#pkl = f"{m_dir}/22019613_efficientnet-b4.pkl"
pkl = f"{m_dir}/22019613_tta3_efficientnet-b4.pkl"
pred = pickle.load(open(pkl, "rb"))
preds.append(pred.values)

#pkl = f"{m_dir}/22019632_timm-resnest101e.pkl"
pkl = f"{m_dir}/22019632_tta3_timm-resnest101e.pkl"
pred = pickle.load(open(pkl, "rb"))
preds.append(pred.values)

#pkl = f"{m_dir}/22019631_tta3_se_resnext101.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{m_dir}/22019717_tta3_timm-resnest101e.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{m_dir}/22019720_tta3_efficientnet-b4.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)

#pkl = f"{m_dir}/ex02C_efficientnet-b4.pkl"
#pkl = f"{m_dir}/ex02C_tta3_efficientnet-b4.pkl"
#pkl = f"{m_dir}/ex02C_tta5_efficientnet-b4.pkl"
#pred = pickle.load(open(pkl, "rb"))
#preds.append(pred.values)
# --------------------------------------------------------------------------------------------

df = pd.read_csv(f"{path}/train.csv")
y = df["label"].values

# 過去コンペのデータは除く
preds = [p[:21397] for p in preds]


if is_del_df_noise_image_id:
    # --- LB0.905の5モデルの内3モデルすべて同じ予測で間違ってるレコード削除 ---    
    preds, y = del_df_noise_image_id(preds, y, not_noise_image_id)

    
if pseudo_th > 0.0 and len(test_df) > 1:
    # --- スコア pseudo_th 以上のサンプルだけ pseudo label に採用する ---
    test_df_orig = test_df.copy()
    test_preds_list_orig = test_preds_list[:]
    test_df['logit'] = test_preds_ensemble.max(1)
    test_df = test_df[test_df['logit'] > pseudo_th]
    test_y = test_df['label'].values
    test_preds_list = [t[test_df['label'].index.values] for t in test_preds_list]
else:
    test_y = test_df['label'].values

    
# クラス数変更
if is_make_noise_model:
    y = np.array([0] * len(y))
    y[noise_idx] = 1
    #test_y = np.array([0] * len(test_y))
    #test_y[test_noise_idx] = 1
print("y..value_counts():")
print(pd.Series(y).value_counts())
    

if s_type == 0:
    # --- pseudo label使わない ---
    x = np.array(preds).transpose(1,2,0)
    x = x.reshape(len(x), 1, StackingCFG.n_classes, len(preds))
    
    oof, oof_loss = train_stacking(x, y, StackingCFG, 
                                   is_check_model=False, 
                                   #noise_idx=noise_idx,
                                  )

elif s_type == 1:
    # --- pseudo labelそのまま突っ込む場合 ---
    preds = [np.vstack((preds[ii], test_preds_list[ii])) for ii in range(len(preds))]
    y = np.concatenate([y, test_y])

    x = np.array(preds).transpose(1,2,0)
    x = x.reshape(len(x), 1, StackingCFG.n_classes, len(preds))
    
    if gauss_scale > 0.0:
        # oofとtestにガウスノイズ加算
        # https://www.kaggle.com/c/stanford-covid-vaccine/discussion/189709
        x += np.random.normal(0.0, scale=gauss_scale, size=x.shape)
    
    oof, oof_loss = train_stacking(x, y, StackingCFG, 
                                   #is_check_model=True, 
                                   is_check_model=False, 
                                   noise_idx=noise_idx,
                                  ) 

elif s_type == 2:
    # --- pseudo labelのデータvalidationには入れない ---
    test_x = np.array(test_preds_list).transpose(1,2,0)
    test_x = test_x.reshape(len(test_x), 1, StackingCFG.n_classes, len(preds))
    
    x = np.array(preds).transpose(1,2,0)
    x = x.reshape(len(x), 1, StackingCFG.n_classes, len(preds))
    
    if gauss_scale > 0.0:
        if is_noise_only_test:
            # testにガウスノイズ加算
            test_x += np.random.normal(0.0, scale=gauss_scale, size=test_x.shape)
        else:
            # oofとtestにガウスノイズ加算
            test_x += np.random.normal(0.0, scale=gauss_scale, size=test_x.shape)
            x += np.random.normal(0.0, scale=gauss_scale, size=x.shape)
            
    oof, oof_loss = train_stacking(x, y, StackingCFG, is_check_model=False, 
                                   add_train_x=test_x, add_train_y=test_y,
                                   noise_idx=noise_idx,
                                  )
    

if pseudo_th > 0.0 and len(test_df) > 1:
    # 抜いたtestデータ戻す
    test_df = test_df_orig
    test_preds_list = test_preds_list_orig
    
    
## predict check
#pred_cnn2d = pred_stacking(x, StackingCFG)
#accuracy_score(y, pred_cnn2d.argmax(1))

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


y..value_counts():
0    20218
1     1179
dtype: int64
x.shape: (21397, 1, 5, 5)
y.shape: (21397,)
---------------------------- seed: 0 ----------------------------
train_idx, valid_idx: 17117, 4280
n_over=10 x, y.shape: (26547, 1, 5, 5) (26547,)



  | Name | Type          | Params
---------------------------------------
0 | net  | CNNStacking2d | 410   
---------------------------------------
410       Trainable params
0         Non-trainable params
410       Total params


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | net  | CNNStacking2d | 410   
---------------------------------------
410       Trainable params
0         Non-trainable params
410       Total params



fold 0 validation loss = 0.48542840802043774
fold 0 validation accuracy = 0.8299065420560747

train_idx, valid_idx: 17117, 4280
n_over=10 x, y.shape: (26547, 1, 5, 5) (26547,)


...

GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | net  | CNNStacking2d | 410   
---------------------------------------
410       Trainable params
0         Non-trainable params
410       Total params



fold 2 validation loss = 0.5266340280462016
fold 2 validation accuracy = 0.8282308950689413

train_idx, valid_idx: 17118, 4279
n_over=10 x, y.shape: (26548, 1, 5, 5) (26548,)


GPU available: True, used: True
TPU available: None, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name | Type          | Params
---------------------------------------
0 | net  | CNNStacking2d | 410   
---------------------------------------
410       Trainable params
0         Non-trainable params
410       Total params



fold 3 validation loss = 0.5249305498067846
fold 3 validation accuracy = 0.8034587520448703

train_idx, valid_idx: 17118, 4279
n_over=10 x, y.shape: (26548, 1, 5, 5) (26548,)



fold 4 validation loss = 0.5037869385889364
fold 4 validation accuracy = 0.7880345875204487

fold 0 mean acc: 0.8183857550123849
 oof: 0.8184
 oof_loss: 0.5094
               precision    recall  f1-score   support

           0       0.96      0.84      0.90     20218
           1       0.14      0.45      0.22      1179

    accuracy                           0.82     21397
   macro avg       0.55      0.65      0.56     21397
weighted avg       0.92      0.82      0.86     21397

 cfm:
 [[16977  3241]
 [  645   534]]
Wall time: 29.3 s


In [12]:
oof

0.8183857550123849

In [13]:
import os, shutil

if os.path.exists("lightning_logs/"):
    shutil.rmtree("lightning_logs/")
    os.mkdir("lightning_logs/")

In [14]:
x.shape

(21397, 1, 5, 5)

In [15]:
frac = 0.4
_y = pd.DataFrame(y)
idx = list(_y[_y[0] == 3].sample(frac=frac).index)
print(len(idx))
_y = _y.drop(index=idx)
print(_y.shape)
x[_y.index].shape
_y[0].values

0
(21397, 1)


array([0, 0, 1, ..., 0, 0, 1])

In [16]:
df = pd.read_csv(f"{path}/train.csv")

x = np.array(preds).transpose(1, 2, 0)
x = x.reshape(len(x), 1, StackingCFG.n_classes, len(preds))

y = df["label"].values
y[y == 3].shape

(13158,)

In [17]:
df = pd.read_csv(f"{path}/train.csv")
y = df["label"].values
y = np.array([0] * len(y))
y[noise_idx] = 1
# if is_noise_model:
#    y = np.array([0] * len(y))
#    y[noise_idx] = 1
## [_ for _ in y]

In [18]:
_y = pd.DataFrame(y)
idx = _y[_y[0] == 1].index
np.concatenate([y, y[idx]])

array([0, 0, 1, ..., 1, 1, 1])